This code is to get data from biznesradar.pl and analyze stocks.

In [7]:
# download libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
import seaborn as sns 

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import plotly as py
py.offline.init_notebook_mode(connected=True)

from plotly.offline import iplot
import plotly as py

# import cufflinks as cf

py.offline.init_notebook_mode(connected=True)
# cf.go_offline()

pd.options.display.max_columns = 100   # set the maximum columns to display
pd.options.display.max_rows = 100   # set the maximum rows to display

In [23]:
# spółka do analizy
Spolka = 'CD-PROJEKT' # podaj nazwę spółki

#Akcjonariat

In [9]:
# table_Akcjonariat = pd.read_csv('https://static.stooq.pl/db/h/d_cw/' + Spolka + '_akcjonariat.csv')
# display(table_Akcjonariat)

HTTPError: HTTP Error 401: Unauthorized

# Przetwarzanie tabel:

### Tabela RZIS

In [ ]:
table_RZS = pd.read_html('https://www.biznesradar.pl/raporty-finansowe-rachunek-zyskow-i-strat/{}'.format(Spolka))

[                                0
0  Zysk netto: wzrost o 3.45% r/r,                                                0
0          Zysk ze sprzedaży: spadek o 7.04% r/r
1     Zysk operacyjny (EBIT): spadek o 3.58% r/r
2  Zysk przed opodatkowaniem: spadek o 4.71% r/r,                                            Unnamed: 0 2007  (gru 07)  \
0                                     Data publikacji     2008-02-07   
1                              Przychody ze sprzedaży        100 206   
2   Techniczny koszt wytworzenia produkcji sprzedanej         94 315   
3                                    Koszty sprzedaży          8 820   
4                             Koszty ogólnego zarządu          7 566   
5                                   Zysk ze sprzedaży        -10 495   
6                      Pozostałe przychody operacyjne          1 187   
7                         Pozostałe koszty operacyjne          7 992   
8                              Zysk operacyjny (EBIT)        -17 300   
9            

AttributeError: 'NoneType' object has no attribute 'head'

In [33]:
table_Bilans = pd.read_html('https://www.biznesradar.pl/raporty-finansowe-bilans/{}'.format(Spolka))
display(table_Bilans).head()

[                                                   0
 0                  Aktywa razem: wzrost o 14.20% r/r
 1                 Aktywa trwałe: wzrost o 16.76% r/r
 2  Rzeczowe składniki majątku trwałego: wzrost o ...
 3               Aktywa obrotowe: wzrost o 11.34% r/r
 4    Inwestycje krótkoterminowe: wzrost o 40.05% r/r
 5  Kapitał własny akcjonariuszy jednostki dominuj...,
                                                    0
 0                        Zapasy: spadek o 56.48% r/r
 1    Należności krótkoterminowe: spadek o 50.84% r/r
 2  Środki pieniężne i inne aktywa pieniężne: spad...,
                                            Unnamed: 0 2007  (gru 07)  \
 0                                     Data publikacji     2008-02-07   
 1                                       Aktywa trwałe          7 729   
 2                     Wartości niematerialne i prawne             77   
 3                                       Wartość firmy              0   
 4                 Rzeczowe składniki m

AttributeError: 'NoneType' object has no attribute 'head'